# RNN Prediction

## Imports and Functions

In [1]:
import pandas as pd

import os
import numpy as np
import sys

import numpy as np
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow_addons.metrics import RSquare


c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
# Add the folder to the Python path

os.chdir("../")
# change working directory to project's root path
print(os.getcwd())
folder_path = os.path.abspath("functions/") #INPUT_PATH)#'path_to_your_folder')  # Replace with the actual folder path
sys.path.insert(0, folder_path)

from Predictions import (
    PredictionExperiment,
    PredictionModel
)

c:\Users\marti\Desktop\data\hw_extra


In [3]:
region="chile"
metadata_path = f"data/climate_features/{region}/metadata.csv"
metadata = pd.read_csv(metadata_path)
metadata.reset_index(inplace=True, drop=True)
display(metadata)

,id,filename,season,indices
0,978f49d7,predictor_978f49d7_1.parquet,1,fde0e327-340e2882-43701738-e306f58b-e601b072-e...
1,978f49d7,predictor_978f49d7_2.parquet,2,fde0e327-340e2882-43701738-e306f58b-e601b072-e...
2,978f49d7,predictor_978f49d7_3.parquet,3,fde0e327-340e2882-43701738-e306f58b-e601b072-e...
3,978f49d7,predictor_978f49d7_4.parquet,4,fde0e327-340e2882-43701738-e306f58b-e601b072-e...
4,978f49d7,predictor_978f49d7_5.parquet,5,fde0e327-340e2882-43701738-e306f58b-e601b072-e...
...,...,...,...,...
247,458d357c,predictor_458d357c_8.parquet,8,32f131d2-69ffcfa8-4af95abb-4a86cb22-52eda853-3...
248,458d357c,predictor_458d357c_9.parquet,9,32f131d2-69ffcfa8-4af95abb-4a86cb22-52eda853-3...
249,458d357c,predictor_458d357c_10.parquet,10,32f131d2-69ffcfa8-4af95abb-4a86cb22-52eda853-3...
250,458d357c,predictor_458d357c_11.parquet,11,32f131d2-69ffcfa8-4af95abb-4a86cb22-52eda853-3...


In [4]:
data = {i: pd.read_parquet(f"data/climate_features/{region}/predictor_9bd58418_{i}.parquet") for i in range(1,13)}
data[12]

,fde0e327-sst,340e2882-sst,b91ccd4e-t,c6184040-max_value,c6184040-latitude,c6184040-longitude,c1c59e4d-u,8c4a4c9c-u,cb76aaa7-u,d416bd24-u,...,PDO-longitude,ONI-PDO,SAM-ONI,DMI-SAM,ADV-DMI,HWN,HWF,HWD,HWM,HWA
0,1.919132,1.738772,1.089508,97.656250,-44.75,284.00,0.105871,-0.078898,-0.430445,-0.884190,...,292.00,-0.37,2.09,-1.38,0.477,0.3,0.9,0.6,0.264685,0.454609
1,-1.787336,-1.410034,-0.630829,373.351562,-30.50,254.75,-0.627693,-0.089662,-0.409054,-0.499015,...,282.00,-0.89,-1.95,2.17,0.047,0.0,0.0,0.0,0.000000,0.000000
2,-0.924454,-1.843321,-0.622803,176.867188,-45.00,267.00,-0.043367,-0.130367,-1.143116,-1.717260,...,292.00,0.27,-0.75,-2.37,-0.360,0.1,0.3,0.3,0.316004,0.459299
3,-1.654869,-1.510971,-0.594147,257.140625,-39.75,245.00,-0.916506,-0.131766,0.242939,2.045603,...,284.50,-1.49,-1.55,0.35,-0.170,0.0,0.0,0.0,0.000000,0.000000
4,-1.731778,-0.329125,-0.561279,-177.140625,-15.00,283.75,-0.382942,0.059941,1.272537,1.059203,...,292.00,1.20,0.86,-5.07,0.028,0.0,0.0,0.0,0.000000,0.000000
5,0.333758,1.009113,1.376251,479.250000,-45.00,279.75,-1.298745,-0.230251,-1.866019,-3.538790,...,292.00,-0.06,0.81,-0.95,-0.020,0.4,1.4,0.8,0.667017,1.287477
6,0.319313,0.542892,1.694794,389.710938,-43.50,284.00,-1.948353,-0.334173,-1.931295,-0.892254,...,285.50,-0.51,-0.08,-0.01,-0.260,0.1,0.3,0.3,0.045834,0.052455
7,0.408862,0.216794,0.038391,1.476562,-16.75,284.00,1.119975,-0.035370,0.270793,-1.237028,...,292.00,0.06,0.52,-1.88,-0.101,0.1,0.3,0.3,0.159879,0.336347
8,-0.305299,-0.773594,0.053955,8.046875,-15.00,284.00,-0.212742,-0.110399,0.689868,0.749291,...,288.75,0.30,0.11,-0.10,-0.483,0.0,0.0,0.0,0.000000,0.000000
9,-1.057992,-0.154225,-0.037231,511.515625,-45.00,262.75,-0.149329,0.024278,-0.341097,-0.983117,...,282.00,0.59,-0.15,2.21,-0.019,0.0,0.0,0.0,0.000000,0.000000


In [5]:
indices_of_interest = ["HWN", "HWF", "HWD", "HWA", "HWM"]
label = ["HWF","HWD"]
#data[12].drop(columns=indices_of_interest, inplace=True)
features = data[12].columns.difference(indices_of_interest)

X = data[12][features]
y = data[12][indices_of_interest]
n_labels = len(indices_of_interest)

n_samples = X.shape[0]
n_features = len(features)
model = PredictionModel(data[12], 12, indices_of_interest, Sequential([
    SimpleRNN(16, activation="tanh", input_shape=(1, n_features)),
    Dropout(0.1),  # Regularization
    Dense(8, activation="relu"),
    Dense(n_labels)  # Predict 5 indices
]), name_regressor="RNN")

In [6]:
model.train_predict(5) ##check how to mesaure metrics correctly because of NN using 5 size output layer

1/1 [==============================] - 0s 14ms/step
[0.09803922 0.10021786 0.10021786 0.10021786 0.10021786 0.10021786
 0.10021786 0.10021786 0.10021786 0.10021786]
2/2 [==============================] - 0s 2ms/step


In [12]:
model.get_metric("mae", "CV")

['RNN',
 12,
 'mae',
 'CV',
 0.5374250984345013,
 0.4159048424597797,
 0.4315235333695399,
 0.5283999179911831,
 0.483200864679509,
 0.47929085138690264]

In [5]:
indices_of_interest = ["HWN", "HWF", "HWD", "HWA", "HWM"]
label = ["HWF","HWD"]
#data[12].drop(columns=indices_of_interest, inplace=True)
features = data[12].columns.difference(indices_of_interest)

X = data[12][features]
y = data[12][indices_of_interest]
n_labels = len(indices_of_interest)

n_samples = X.shape[0]
n_features = len(features)

# Standardize inputs
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Reshape for RNN/LSTM (samples, timesteps=1, features)
X_scaled = X_scaled.reshape((n_samples, 1, n_features))  

# Train/Validation split
X_train, X_val = X_scaled[:46], X_scaled[46:]
y_train, y_val = y_scaled[:46], y_scaled[46:]


In [14]:
# Define an RNN model with Dropout
rnn_model = Sequential([
    SimpleRNN(16, activation="tanh", input_shape=(1, n_features)),
    Dropout(0.1),  # Regularization
    Dense(8, activation="relu"),
    Dense(n_labels)  # Predict 5 indices
])

# Compile
rnn_model.compile(optimizer="adam", loss="mae", metrics=["mse", "mape", RSquare()])

# Early stopping callback
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Train with validation
rnn_model.fit(X_train, y_train, epochs=200, batch_size=8, validation_data=(X_val, y_val),
              callbacks=[early_stopping], verbose=1)

# Summary
rnn_model.summary()


Epoch 1/200
6/6 [==============================] - 3s 79ms/step - loss: 0.8858 - mse: 1.2780 - mape: 247.9996 - r_square: -0.4531 - val_loss: 1.1801 - val_mse: 2.1849 - val_mape: 114.1782 - val_r_square: -0.6159
Epoch 2/200
6/6 [==============================] - 0s 12ms/step - loss: 0.8165 - mse: 1.1508 - mape: 211.7742 - r_square: -0.3085 - val_loss: 1.1671 - val_mse: 2.1323 - val_mape: 111.8369 - val_r_square: -0.5853
Epoch 3/200
6/6 [==============================] - 0s 12ms/step - loss: 0.8207 - mse: 1.1745 - mape: 217.6633 - r_square: -0.3386 - val_loss: 1.1553 - val_mse: 2.0857 - val_mape: 109.4768 - val_r_square: -0.5601
Epoch 4/200
6/6 [==============================] - 0s 12ms/step - loss: 0.7639 - mse: 1.0600 - mape: 187.5799 - r_square: -0.2058 - val_loss: 1.1467 - val_mse: 2.0502 - val_mape: 107.4829 - val_r_square: -0.5408
Epoch 5/200
6/6 [==============================] - 0s 12ms/step - loss: 0.7585 - mse: 1.0268 - mape: 202.1478 - r_square: -0.1629 - val_loss: 1.1404 - v

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
rnn_model = Sequential([
    SimpleRNN(16, activation="tanh", input_shape=(1, n_features)),
    Dropout(0.1),  # Regularization
    Dense(8, activation="relu"),
    Dense(n_labels)  # Predict 5 indices
])
def reshape_for_keras(X):
    return np.expand_dims(X, axis=1)
tscv = TimeSeriesSplit(test_size=5)
X = data[12][features]
y = data[12][indices_of_interest]
r2_tscv = []
mape_tscv= []
train_sizes = []
for train_index, test_index in tscv.split(X):
    train_sizes.append(len(train_index))
    train_data, test_data, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    X_train = reshape_for_keras(train_data)
    X_test = reshape_for_keras(test_data)
    rnn_model.compile(optimizer="adam", loss="mae")
    rnn_model.fit(X_train, y_train, epochs=200, batch_size=8, verbose=0, validation_data=(X_test, y_test))
    pred = rnn_model.predict(X_test)
    r2_tscv.append(r2_score(y_test, pred, multioutput="raw_values")) 
    mape_tscv.append(mean_absolute_percentage_error(y_test, pred, multioutput="raw_values"))
weights = np.array(train_sizes)/np.sum(train_sizes)
r2_tscv = np.dot(np.array(r2_tscv).T,weights)
mape_tscv = np.dot(np.array(mape_tscv).T,weights)

1/1 [==============================] - 0s 92ms/step
[array([0.70361004, 0.68047515, 0.72400961, 0.64392887, 0.63689179]), array([ -1.05026093,  -4.60477475,  -1.20002839, -15.7154569 ,
       -14.37387745]), array([0.71218407, 0.92948281, 0.88717301, 0.8204686 , 0.84586961]), array([0.66614026, 0.43188497, 0.45762006, 0.32655183, 0.52341476]), array([ 0.32980692,  0.3549644 ,  0.25480821, -0.40912829, -0.52822707])]


In [15]:
r2_tscv = np.dot(r2_tscv, weights)
r2_tscv

array([ 0.29920704, -0.3197702 ,  0.24469555, -2.479619  , -2.23010082])

In [15]:
# Define an LSTM model with Dropout
lstm_model = Sequential([
    LSTM(16, activation="tanh", input_shape=(1, n_features)),
    Dropout(0.1),  # Regularization
    Dense(8, activation="relu"),
    Dense(n_labels)  # Predict 5 indices
])

# Compile
lstm_model.compile(optimizer="adam", loss="mae", metrics=["mse", "mape", RSquare()])

# Train with validation
lstm_model.fit(X_train, y_train, epochs=200, batch_size=8, validation_data=(X_val, y_val),
               callbacks=[early_stopping], verbose=1)

# Summary
lstm_model.summary()


Epoch 1/200
6/6 [==============================] - 3s 100ms/step - loss: 0.7473 - mse: 0.9057 - mape: 146.3605 - r_square: -0.0233 - val_loss: 1.1267 - val_mse: 1.9693 - val_mape: 97.3572 - val_r_square: -0.5427
Epoch 2/200
6/6 [==============================] - 0s 10ms/step - loss: 0.7334 - mse: 0.8930 - mape: 126.6799 - r_square: -0.0089 - val_loss: 1.1252 - val_mse: 1.9685 - val_mape: 96.6118 - val_r_square: -0.5421
Epoch 3/200
6/6 [==============================] - 0s 10ms/step - loss: 0.7214 - mse: 0.8768 - mape: 132.0839 - r_square: 0.0090 - val_loss: 1.1234 - val_mse: 1.9657 - val_mape: 95.9315 - val_r_square: -0.5398
Epoch 4/200
6/6 [==============================] - 0s 10ms/step - loss: 0.7118 - mse: 0.8699 - mape: 124.9203 - r_square: 0.0173 - val_loss: 1.1223 - val_mse: 1.9689 - val_mape: 95.2261 - val_r_square: -0.5422
Epoch 5/200
6/6 [==============================] - 0s 9ms/step - loss: 0.7018 - mse: 0.8511 - mape: 125.4212 - r_square: 0.0391 - val_loss: 1.1221 - val_mse:

In [8]:
lstm_model.get_metrics_result()

{'loss': <tf.Tensor: shape=(), dtype=float32, numpy=2.0460103>,
 'mae': <tf.Tensor: shape=(), dtype=float32, numpy=1.1040316>,
 'mape': <tf.Tensor: shape=(), dtype=float32, numpy=84.136955>,
 'r_square': <tf.Tensor: shape=(), dtype=float32, numpy=-0.59095705>}